In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.externals import joblib
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import display, HTML

# svm, knc
classifier = 'svm'

def input(file):
    data = pd.read_csv(file)
    data['window'] = (pd.to_datetime(data['dep_date'], format="%Y%m%d") - pd.to_datetime(data['booking_date'], format="%Y%m%d")).dt.days 
    data['dep_day'] = pd.to_datetime(data['dep_date'], format="%Y%m%d").dt.dayofweek
    data["pax"] = data["pax"].apply(np.log)
    
    time_bins = [0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400]
    time_bin_labels = [1,2,3,4,5,6,7,8]
    
    data['dep_time'] = pd.cut(data['dep_time'], bins=time_bins, labels=time_bin_labels).cat.codes
    X = pd.get_dummies(data, columns=['origin', 'destination'])
    
    return X

data = input('input/test.csv')
X = data
_id = data['id']
del X['id']
del data['dep_date']

model_pkl = open('models/' + classifier + '.2.pkl', 'rb')
model = pickle.load(model_pkl)
predict = model.predict(X)
predicted_proba = model.predict_proba(X)

result = pd.DataFrame(predicted_proba, columns=model.classes_)
result.insert(loc=0, column='id', value=_id)
display(HTML(result.head(100).to_html()))

,id,Classic,Deal,Flex,Saver
0,73831,0.021617,0.828152,0.003118,0.147113
1,73832,0.028880,0.297980,0.005448,0.667692
2,73833,0.019603,0.830300,0.002894,0.147203
3,73834,0.044549,0.027961,0.864207,0.063282
4,73835,0.049428,0.031653,0.008383,0.910536
5,73836,0.017928,0.830933,0.002678,0.148461
6,73837,0.108647,0.045473,0.793914,0.051966
7,73838,0.425889,0.034682,0.477414,0.062015
8,73839,0.842478,0.015596,0.079014,0.062912
9,73840,0.621948,0.017790,0.022311,0.337951


In [2]:
result.to_csv('result/' + classifier + 'test.csv', sep=',', encoding='utf-8')